In [1]:
import jupyterlab_dash
import dash
import dash_html_components as html

import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
import datetime
warnings.filterwarnings('ignore')

import dash
import dash_core_components as dcc
import dash_html_components as dhc
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import flask

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred

In [2]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

sql="""SELECT s.ticker, max(d.symbol_id), max(d.date_price)
FROM public.symbol s inner join d_data d on s.id= d.symbol_id
group by s.ticker"""
df_ticker_last_day=pd.read_sql(sql,con=conn_secmaster)

def level_plot(df):
    support_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_support']]
    resistance_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_resistance']]
    end_dt=df.index[-1]
    res_plot_ls=[]
    sup_plot_ls=[]
    for res in resistance_ls[:5]:
        res_plot_ls.append(dict(x0=res[2],x1=end_dt,y0=res[0],y1=res[1],yref='y1',opacity=.2,fillcolor='Red',line=dict(color="black",width=1)))
    for sup in support_ls[:5]:
        sup_plot_ls.append(dict(x0=sup[2],x1=end_dt,y0=sup[0],y1=sup[1],yref='y1',opacity=.2,fillcolor='green',line=dict(color="black",width=1)))
    return (res_plot_ls+sup_plot_ls)

def data_selector(symbol_id):
    sql="select * from indicator"
    ind_list=list(pd.read_sql(sql,con=conn_indicator)['name'])
#     symbol_id='EUR_USD'#'AAPL'#'BOM500114'
    start_date=datetime.datetime(2018,1,1).strftime("%Y-%m-%d")
#     indicator_name = 'candle_1'
    
    sql="select d.date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from d_data d join symbol s on d.symbol_id = s.id where s.ticker='%s' and d.date_price > '%s'" %(symbol_id, start_date)
    df_price=pd.read_sql(sql,con=conn_secmaster)
    
    df_all_ind=pd.DataFrame()
    for ind in ind_list:
        print(ind)
        sql="select d.date_price as date, d.value from d_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, ind, start_date)
        df_indicator=pd.read_sql(sql,con=conn_indicator)
        df_indicator.set_index('date',inplace=True)
        df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
        df_indicator.columns=[ind+"_"+col for col in df_indicator.columns]
        if df_all_ind.empty:
            df_all_ind=pd.merge(left=df_price, right=df_indicator,on='date')
        else:
            df_all_ind=pd.merge(left=df_all_ind, right=df_indicator,on='date')
    
    df_all_ind.set_index('date',inplace=True)
    return df_all_ind

In [4]:
viewer = jupyterlab_dash.AppViewer()

# app = dash.Dash(__name__)

# app.layout = html.Div('Hello World Poda')

# viewer.show(app)
external_stylesheets = ['https://codepen.io/amyoshino/pen/jzXypZ.css']

dfPool = pd.read_csv('6_ml_log.csv')
# to display first item when page loaded
currencyPairs = list(set(dfPool[dfPool.columns[0]])) #reading distict currency pairs/security
df = (dfPool[dfPool['security'] == currencyPairs[0]]) #reading first security data for initial rendering

# server = flask.Flask(__name__)
app = dash.Dash(
    __name__,
#     server=server,
    external_stylesheets=external_stylesheets
)



# indicators = []
# for col in df.columns:
#     if (col not in ['security', 'datetime', 'open', 'high', 'low', 'close']):
#         indicators.append(col)

# indicatorMap = [1 for x in indicators] #for mapping into rowsize vector for allocating height

# try:
#     indicatorMap.remove(1)  #for removing an item that represents candle stick
# except ValueError:
#     pass

# Currentfig = make_subplots(
#     rows=len(indicators), cols=1, shared_xaxes=True, vertical_spacing=0.02
# )

# Currentfig.add_trace(go.Candlestick(x=df['datetime'],
#                                     open=df['open'], high=df['high'],
#                                     low=df['low'], close=df['close']))

##########################################
app.layout = dhc.Div([
    dhc.Div([
        dcc.Graph(id="plot-candle")#,figure=Currentfig
    ],style = {'display': 'inline-block', 'width': '100%','height':'200%'},className='row'),
    dhc.Div(
        [
            dhc.Div([dcc.Dropdown(id='dropdown-securities',
                    options=[{'label': i, 'value': i} for i in df_ticker_last_day['ticker'].unique()], multi=False, value="EUR_USD")],className='six columns')
#             ,dhc.Div([dcc.Dropdown(id='dropdown-indicators',
#                     multi=True,options=[{'label': i, 'value': i} for i in indicators])],className='six columns')
                     
    ],className='row'),
])

@app.callback(
    Output('plot-candle', 'figure'),
    [Input('dropdown-securities', 'value')]
)
def updatePlot(securityValue):
    
    interested_feature='anomaly_vol_anomaly'
    df=data_selector(securityValue)
    df_candle_1=df[-10:]
    df_candle_1=df_candle_1[df_candle_1['candle_1_pattern_name']!='']
    df_candle_1['pattern']='1'

    df_candle_2=df[-10:]
    df_candle_2=df_candle_2[df_candle_2['candle_2_pattern_name']!='']
    df_candle_2['pattern']='2'

    df_candle_3=df[-10:]
    df_candle_3=df_candle_3[df_candle_3['candle_3_pattern_name']!='']
    df_candle_3['pattern']='3'
    
    data = [ dict(
    type = 'candlestick',
    open = df.open,
    high = df.high,
    low = df.low,
    close = df.close,
    x = df.index,
    yaxis = 'y1',
    name = 'price'
    )]
    

    data.append( dict( x=df.index, y=df.volume,                         
                             marker=dict( color='blue' ),
                             type='bar', yaxis='y2', name='Volume'))

    data.append( dict( x=df.index, y=df[interested_feature],                         
                             marker=dict( color='red' ),
                             type='scatter', yaxis='y3', name=interested_feature))

    data.append( dict( x=df_candle_1.index, y=df_candle_1['high'],
                             text=df_candle_1['pattern'],
                             mode="markers+text",textfont_size=15,textposition="top center",
                             marker=dict( color='blue' ),
                             type='scatter', yaxis='y1', name='candle_patter_1'))

    data.append( dict( x=df_candle_2.index, y=df_candle_2['high'],
                             text=df_candle_2['pattern'],
                             mode="markers+text",textfont_size=15,textposition="top center",
                             marker=dict( color='red' ),
                             type='scatter', yaxis='y1', name='candle_patter_2'))


    data.append( dict( x=df_candle_3.index, y=df_candle_3['high'],
                             text=df_candle_3['pattern'],
                             mode="markers+text",textfont_size=15,textposition="top center",
                             marker=dict( color='red' ),
                             type='scatter', yaxis='y1', name='candle_patter_3'))


    layout=dict()    
    layout['xaxis'] = dict( rangeslider = dict( visible = False ),autorange=True,fixedrange=False,visible=False,type='category')#type='category',
    layout['yaxis'] = dict( domain = [0.2, 1],autorange = True,fixedrange=False)
    layout['yaxis2'] = dict( domain = [0.0, 0.1],autorange = True,fixedrange=False)
    layout['yaxis3'] = dict( domain = [0.1, 0.2],autorange = True,fixedrange=False)
    layout['shapes'] = level_plot(df)
    layout['margin']=dict(l=20, r=10)
    layout['paper_bgcolor']="LightSteelBlue"
    layout['width']=2200
    layout['height']=1000
    
    


    fig = dict( data=data, layout=layout )
    return fig


@app.callback(
    Output(component_id='plot_candle', component_property='figure'),
    [Input('plot_candle', 'relayoutData')],
    [State('plot_candle', 'figure')]
)
def update_output_div(input_value, fig):
    fig['layout'] = {"title": "Poda Patti"}
    print(input_value)
    return fig

In [5]:
viewer.show(app)

Dash is running on http://localhost:35083/proxy/35083/

Dash is running on http://localhost:35083/proxy/35083/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

